### Extending Cities Table with Pandas

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
# Load two different tables we previously got from Wikipedia using WebScraper
# URL: "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
df_cities = pd.read_csv('~/Desktop/Webscraper_Topos_Challenge/output/cities/cities.csv')
# URL: "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_area"
df_areas = pd.read_csv('~/Desktop/Webscraper_Topos_Challenge/output/areas/areas.csv')

In [3]:
df_cities.head(5)

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City,New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston,Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


In [4]:
df_areas.head(5)

,Rank,City,State,Land area (sq mi),Land area (km2),Water area (sq mi),Water area (km2),Total area (sq mi),Total area (km2),Population (2010)
0,1,Sitka,Alaska,"2,870.3","7,434","1,941.0","5,027","4,811.4","12,461","8,881"
1,2,Juneau,Alaska,"2,701.9","6,998",552.0,"1,430","3,253.9","8,428","31,275"
2,3,Wrangell,Alaska,"2,541.5","6,582",920.6,"2,384","3,462.1","8,967","2,369"
3,4,Anchorage,Alaska,"1,704.7","4,415",256.3,664,"1,961.0","5,079","291,826"
4,5,Jacksonville,Florida,747.0,"1,935",127.6,330,874.6,"2,265","821,784"


In order to include data from another source to our initial "df_cities" table we can do a join with "df_areas". However, before doing so we need a "key" to uniquely identify each city. I will consider the pair "City" and "State" columns as a unique identifier for any given city:

Note I avoided using just the city name as a unique identifier. For instance, 34 States have a city named Springfield in the US. This might seem irrelevant in this toy example but could be important to avoid meaningless matches if we were, for example, to scale the problem to "Cities Around the World".

1) Take the City and State columns from both tables as a common unique identifier 

2) Transform every character to lower case and remove spaces to standarize our keys.

3) Perfom the join. I will choose left join, with "df_cities" on the left. Here I am assuming that "df_areas" information is not relevant if any pair of keys is not present in "df_cities".

In [5]:
# Add a join_key column according to the instructions above
df_cities = df_cities.assign(city_key = df_cities.apply(lambda row: str.lower(str(row['City'])).replace(' ', ''), axis = 1))
df_areas = df_areas.assign(city_key = df_areas.apply(lambda row: str.lower(str(row['City'])).replace(' ', ''), axis = 1))

df_cities = df_cities.assign(state_key = df_cities.apply(lambda row: str.lower(str(row['State'])).replace(' ', ''), axis = 1))
df_areas = df_areas.assign(state_key = df_areas.apply(lambda row: str.lower(str(row['State'])).replace(' ', ''), axis = 1))


# Example row
df_cities.iloc[0,:]
df_areas.iloc[0,:]


Rank                        1
City                    Sitka
State                  Alaska
Land area (sq mi)     2,870.3
Land area (km2)         7,434
Water area (sq mi)    1,941.0
Water area (km2)        5,027
Total area (sq mi)    4,811.4
Total area (km2)       12,461
Population (2010)       8,881
city_key                sitka
state_key              alaska
Name: 0, dtype: object

In [10]:
# Perfom Left Join
df_merge = df_cities.merge(df_areas, left_on = ['city_key','state_key'] , right_on =['city_key','state_key'], how = 'left')

# Clean resulting table by dropping any duplicate or redundant column
df_merge.drop(['City_y', 'State_y', '2016 land area.1', '2016 population density.1', 'Population (2010)'], inplace = True, axis = 1)
df_merge.head(5)

# Extract latitude 
import re



In [7]:
# Count number of non-null extensions after join
print('Total number of rows with extended data:',df_merge['Rank'].count())

# Total number of rows on DataFrame
print('Total number of rows in table:', df_merge['City_x'].count())

Total number of rows with extended data: 103
Total number of rows in table: 314


### Data Visualization

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gdp
from shapely.geometry import Point, Polygon

%matplotlib inline


ModuleNotFoundError: No module named 'geopandas'

In [89]:
# Save everything to .csv file
df_merge.to_csv('topos_table.csv', index = True)